In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from config import gkey
gmaps.configure(api_key=gkey)

In [2]:
cities = pd.read_csv("weather_df.csv")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,mahebourg,40,MU,77,-20.41,57.70,69.80,6.93
1,1,poum,0,NC,57,-20.23,164.02,70.48,12.24
2,2,provideniya,90,RU,76,64.38,-173.30,55.40,4.47
3,3,albany,100,US,80,42.60,-73.97,70.00,10.89
4,4,ushuaia,0,AR,74,-54.80,-68.30,28.40,4.70


In [3]:
humidity = cities["Humidity"]
maxhumidity = humidity.max()
Coordinates = cities[["Latitude", "Longitude"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(Coordinates, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
narrowed_city_df = cities.loc[(cities["Max Temp"] > 80) & (cities["Max Temp"] < 90) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.loc[(cities["Wind Speed"] < 7), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')

narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df

#I've chosen somewhere hotter than 80 and almost windless

,Unnamed: 0,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,11,sakakah,0,SA,15,29.97,40.21,86.00,5.82
1,14,borogontsy,0,RU,35,62.67,131.17,84.88,5.57
2,248,sibolga,0,ID,63,1.74,98.78,82.85,0.51
3,250,talbahat,0,IN,57,25.05,78.43,89.55,5.35
4,251,waddan,0,LY,27,29.16,16.14,81.46,4.79
5,264,beidao,0,CN,47,34.57,105.89,82.99,3.85
6,335,vostok,0,RU,49,46.49,135.88,80.29,6.64
7,353,manokwari,0,ID,63,-0.87,134.08,84.99,4.90
8,368,burhanpur,0,IN,76,21.30,76.23,82.89,5.06
9,445,awbari,0,LY,20,26.59,12.78,81.28,5.46


In [6]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Latitude']
    lng = narrowed_city_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,11,sakakah,0,SA,15,29.97,40.21,86.00,5.82,Sakaka
1,14,borogontsy,0,RU,35,62.67,131.17,84.88,5.57,Borogontsy
2,248,sibolga,0,ID,63,1.74,98.78,82.85,0.51,Sibolga
3,250,talbahat,0,IN,57,25.05,78.43,89.55,5.35,Talbehat
4,251,waddan,0,LY,27,29.16,16.14,81.46,4.79,Waddan


In [7]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [8]:
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))